In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../../data/df_segmentation.csv", index_col=[0])
df["date_mensuelle"] = pd.to_datetime(df["date_mensuelle"])

# MOC C

In [3]:
resultats = df.groupby("Classes").agg(moyenne_TARGET=("TARGET", "mean")).to_dict()["moyenne_TARGET"]

In [4]:
resultats

{1: 0.17558199689601656,
 2: 0.11039856923863056,
 3: 0.0796209282499653,
 4: 0.06726292921028129,
 5: 0.050026666666666664,
 6: 0.0385176539247155,
 7: 0.03720294586591755}

In [5]:
def Boostrapping_Classes(classe):
    df_classe = df[df['Classes'] == classe]
    tx_defaut_liste = []

    for _ in range(1000):
        echantillon = df_classe["TARGET"].sample(n=len(df_classe), replace=True)
        tx_defaut_liste.append(echantillon.mean())
        
    tx_defaut_liste = np.array(tx_defaut_liste)

    decile9 = np.percentile(tx_defaut_liste, 90)

    MOC_C = decile9 - resultats[classe]
    
    return(classe, resultats[classe], decile9, MOC_C)

In [6]:
MOC_C = pd.DataFrame(columns = ["Classe", "LRA", "Décile9", "Moc_C"])

In [7]:
for i in range(1,8):
    classe, LRA, decile9, MOC_C_classe = Boostrapping_Classes(i)
    MOC_C.loc[len(MOC_C)] = [classe, LRA, decile9, MOC_C_classe]

In [8]:
MOC_C

,Classe,LRA,Décile9,Moc_C
0,1.0,0.175582,0.180341,0.004759
1,2.0,0.110399,0.111884,0.001486
2,3.0,0.079621,0.081227,0.001606
3,4.0,0.067263,0.068406,0.001143
4,5.0,0.050027,0.051200,0.001173
5,6.0,0.038518,0.040050,0.001532
6,7.0,0.037203,0.039329,0.002126


# MOC A
### Valeurs manquantes dans les variables explicatives

In [9]:
var_explicatives = ['REGION_RATING_CLIENT_W_CITY', 'DAYS_CREDIT_ENDDATE', 'RATE_DOWN_PAYMENT', 'AMT_PAYMENT', 'NAME_INCOME_TYPE', 'OCCUPATION_TYPE']

In [11]:
train = pd.read_csv("../../data/application_train_vf.csv")
df_bur = pd.read_csv('../../data/bureau.csv')
df_bur_group = df_bur[['DAYS_CREDIT_ENDDATE', 'SK_ID_CURR']].groupby('SK_ID_CURR').sum()
df_bur_group.reset_index(inplace=True)

df_prev = pd.read_csv('../../data/previous_application.csv')
df_prev_group = df_prev[['SK_ID_CURR', 'DAYS_FIRST_DRAWING', 'RATE_DOWN_PAYMENT']].groupby(
    'SK_ID_CURR').sum()
df_prev_group.reset_index(inplace=True)

df_ins = pd.read_csv('../../data/installments_payments.csv')
df_ins_group = df_ins[['SK_ID_CURR', 'AMT_PAYMENT']].groupby('SK_ID_CURR').sum()
df_ins_group.reset_index(inplace=True)

train = train.merge(df_bur_group, on='SK_ID_CURR', how='left')
train = train.merge(df_prev_group, on='SK_ID_CURR', how='left')
train = train.merge(df_ins_group, on='SK_ID_CURR', how='left')

In [12]:
train[var_explicatives].isna().sum()

REGION_RATING_CLIENT_W_CITY        0
DAYS_CREDIT_ENDDATE            43764
RATE_DOWN_PAYMENT              16395
AMT_PAYMENT                    15810
NAME_INCOME_TYPE                   0
OCCUPATION_TYPE                96049
dtype: int64

### Période Manquante

In [13]:
max_year = df["date_mensuelle"].dt.year.max()
min_year = df["date_mensuelle"].dt.year.min()

In [14]:
periodes_attendues = [f"{year}_{quarter}" for year in range(min_year, max_year) for quarter in range(1, 5)]
periodes_reelles = df['date_trimestrielle'].unique()

periodes_manquantes = set(periodes_attendues) - set(periodes_reelles)
print("Périodes manquantes dans les données :", periodes_manquantes)

Périodes manquantes dans les données : set()


### Crise Covid

In [15]:
df['periode_crise_covid'] = ((df['date_trimestrielle'] == '2020_1') | (df['date_trimestrielle'] == '2020_2'))

sans_covid = df[df['periode_crise_covid'] == False].copy()

In [16]:
Covid_défaut = pd.DataFrame(columns = ["Classe", "Taux avec Covid", "Taux sans Covid", "Marge Conservatisme (%)"])

In [17]:
for i in range(1,8):
    tx_ss_covid = sans_covid[sans_covid["Classes"]==i]["TARGET"].mean()
    marge = ((tx_ss_covid/resultats[i])-1)*100
    Covid_défaut.loc[len(Covid_défaut)] = [i, resultats[i], tx_ss_covid, marge]

In [18]:
Covid_défaut

,Classe,Taux avec Covid,Taux sans Covid,Marge Conservatisme (%)
0,1.0,0.175582,0.175284,-0.169793
1,2.0,0.110399,0.110143,-0.231685
2,3.0,0.079621,0.079729,0.136291
3,4.0,0.067263,0.067335,0.106997
4,5.0,0.050027,0.050113,0.172120
5,6.0,0.038518,0.038307,-0.546085
6,7.0,0.037203,0.037127,-0.204379
